
![Image Description](/images/Operations_Research_Methodology_Diagram.png)







I've been working on a workforce scheduling problem using linear programming in Python with the PuLP library. I'm trying to assign employees to different brands, ensuring certain conditions are met. Here's a detailed explanation of the problem and the code I've implemented:

Problem Statement:
Assign a fixed number of employees to different brands.
Ensure that an employee works on only one brand per day.
An employee can work a maximum of 9 hours and a minimum of 5 hours consecutively.
Need to fulfill staffing requirements for each brand.

In [7]:
import itertools

from pulp import LpProblem, LpVariable, LpBinary, lpSum

n_weeks = 2
days_per_week = 7
total_days = n_weeks * days_per_week
shop_open = 10  # 10 AM
shop_close = 20  # 8 PM
min_working_hours = 5
max_working_hours = 9
n_employees = 25

### DATA

# 5-day or 4-day contract type (60% each for now)
five_days_count = n_employees * 0.6

brands = {
    "PHONE": ["PHONE1", "PHONE2"],
    "TV": ["TV1", "TV2", "TV3"]
}
brand_items = tuple(itertools.chain(*brands.values()))

# employee table  e# : ( 4/5 day , [brand_item, ...] )
employees = {f'e{i}': (5*n_weeks if i < five_days_count else 4*n_weeks, [brand_items[i % len(brand_items)]]) for i in range(n_employees)}
# print(employees)

staffing_requirements = {
    "PHONE1": 1,
    "PHONE2": 1,
    "TV1": 1,
    "TV2": 1,
    "TV3": 1}

# the hours on which it is possible to commence a 5-hr shift
poss_starts = [hr for hr in range(shop_open, shop_close - min_working_hours + 1)]

prob = LpProblem('shift_sked')

### SETS / INDEXES

hours = list(range(shop_open, shop_close))  # convenience for readability...
days = list(range(total_days))
#                                                     Domain trim-down:  vvvv                 vvvv
EDHB_employees = {(e, d, h, b) for e in employees for d in days for h in poss_starts for b in employees[e][1]}
EDHB_brand_items = {(e, d, h, b) for e in employees for d in days for h in hours for b in employees[e][1]}

### VARS

start_shift = LpVariable.dicts('start', indices=EDHB_employees, cat=LpBinary) # e starts shift on day d, hour h, for brand item b
covers = LpVariable.dicts('covers', indices=EDHB_brand_items, cat=LpBinary)  # e covers brand item b on day d, hour h
max_shifts = LpVariable('max_shifts')  # the max shifts by any employee

### OBJ:  Minimize shift starts
prob += lpSum(start_shift)

# alternate objectives for experimenting...
# prob += lpSum(covers)  # should produce days*hours*requirements
# prob += max_shifts  # the max number of shifts by any employee  NOTE:  This is tougher solve, longer...

### CONSTRAINTS

# 1 & 2.  Limit shift starts by 4/5 day limit, and can only start 1 shift/day
for e in employees:
    prob += sum(start_shift[e, d, h, b] for d in days for h in poss_starts for b in employees[e][1]) <= employees[e][0]
    for d in days:
        prob += sum(start_shift[e, d, h, b] for h in poss_starts for b in employees[e][1]) <= 1

# 3. Link coverage to shift starting
for (e, d, h, b) in EDHB_brand_items:
    if b not in employees[e][1]:  # this employee cannot 'cover' this item
        prob += covers[e, d, h, b] <= 0
    elif h in poss_starts:  # a start or continued coverage can work...
        prev_hour = covers[e, d, h-1, b] if h > shop_open else None
        prob += covers[e, d, h, b] <= start_shift[e, d, h, b] + prev_hour
    else:  # only previous hour coverage can work (start not possible)
        prob += covers[e, d, h, b] <= covers[e, d, h-1, b]

# 4.  min/max coverage
for e in employees:
    for d in days:
        starts_shift = sum(start_shift[e, d, h, b] for h in poss_starts for b in employees[e][1])
        prob += sum(covers[e, d, h, b] for h in hours for b in employees[e][1]) <= max_working_hours * starts_shift
        prob += sum(covers[e, d, h, b] for h in hours for b in employees[e][1]) >= min_working_hours * starts_shift

# 5.  brand-item coverage minimums
for (d, h, b) in itertools.product(days, hours, brand_items):
    prob += sum(covers[e, d, h, b] for e in employees if b in employees[e][1]) >= staffing_requirements[b]

# 6.  Capture max shifts
for e in employees:
    prob += max_shifts >= sum(start_shift[e, d, h, b] for d in days for h in poss_starts for b in employees[e][1])

# print(prob)

cbc_path = '/opt/homebrew/opt/cbc/bin/cbc'
solver = pulp.COIN_CMD(path=cbc_path)
res = prob.solve(solver)

# highs_path = '/opt/homebrew/bin/highs'
# solver_2 = pulp.HiGHS_CMD(path=highs_path)
# res = prob.solve(solver_2)

NameError: name 'pulp' is not defined

In [10]:
import pandas as pd
from datetime import datetime, timedelta, time
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

file_path = r'C:\Users\Alvaro\Documents\Facultad\MBZUAI\Internship\Etihad\Internship\Zonal Allocation\Automated allocations\Automated Allocation Excel v6 - corrected dates.xlsm'
# Load data
manpower_df = pd.read_excel(file_path, sheet_name='Manpower').head(30)
flights_df = pd.read_excel(file_path, sheet_name='Flights').head(30)


# Let's handle the NaN values
manpower_df['Main Certifications'] = manpower_df['Main Certifications'].apply(lambda x: [] if pd.isna(x) else x)
manpower_df['Cat-A Certifications'] = manpower_df['Cat-A Certifications'].apply(lambda x: [] if pd.isna(x) else x)


In [11]:
# Extract certifiers, aircraft, bays, and shifts from the data
certifiers = manpower_df['Name'].unique()
aircraft_wp = flights_df['WP'].unique()
bays = flights_df['Bay'].unique()

# Create dictionaries for certifier and aircraft information
certifier_info = manpower_df.set_index('Name').to_dict('index')
aircraft_info = flights_df.set_index('WP').to_dict('index')

# Define what a long stop is, in minutes 
long_stop = 5*60

In [12]:
########################### COSTS ########################################

# Function to calculate movement cost (or time) between bays
# If they are of the same area (for instance, both are 600), I define the movement cost as 2 minutes per bay, with a maximum of 10 minutes
# If they are of different areas, I set the movement cost as 30 minutes
def calculate_movement_cost(bay1, bay2):
    if bay1 // 100 == bay2 // 100:
        return min(2 * abs(bay1 - bay2), 10)
    else:
        return 30

# Create a movement time matrix. This matrix will be for all the possible pairs of aircraft
movement_cost_matrix = {}
for a1 in aircraft_wp:
    for a2 in aircraft_wp:
        movement_cost_matrix[(a1, a2)] = calculate_movement_cost(aircraft_info[a1]['Bay'], aircraft_info[a2]['Bay'])

# We can also define a cost of changing bay zones.
def calculate_zone_change_cost(bay_zone_1, bay_zone_2):
    if bay_zone_1 != bay_zone_2:
        return 100
    else:
        return 0


# Cost of assigning a Cat-A certifier instead of a Main certifier
def calculate_certification_cost(certifier, aircraft):
    if (
        aircraft_info[aircraft]["Aircraft type"]
        in certifier_info[certifier]["Main Certifications"]
    ):
        return 0
    elif (
        aircraft_info[aircraft]["Aircraft type"]
        in certifier_info[certifier]["Cat-A Certifications"]
    ):
        return 50


# Cost of assigning a shift leader
def calculate_shift_leader_cost(certifier):
    if certifier_info[certifier]['Type'] == 'LE':
        return 100
    else:
        return 0

# Cost of assigning a quality control certifier
def calculate_quality_control_cost(certifier):
    if certifier_info[certifier]['Primary Bay Zone'] == 'Quality Control':
        return 200
    else:
        return 0

# Define the cost for assigning multiple aircraft to the same certifier
# I think this cost should be more incremental, like 2*number of aircrafts
def calculate_assignment_cost(num_aircraft):
    return 2*num_aircraft * 50

In [13]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus

# Initialize the problem
problem = LpProblem("Aircraft_Assignment", LpMinimize)

# Define decision variables
assignment = LpVariable.dicts("Assign", (certifiers, aircraft_wp), 0, 1, cat='Binary')

# Auxiliary variables for movement
movement = LpVariable.dicts("Movement", (certifiers, aircraft_wp, aircraft_wp), 0, 1, cat='Binary')

# Calculate total costs
total_cost = lpSum([
    movement_cost_matrix[(a1, a2)] * movement[c][a1][a2]
    for c in certifiers for a1 in aircraft_wp for a2 in aircraft_wp
])

# Add the cost of assigning multiple aircraft to the same certifier
for c in certifiers:
    num_assigned_aircraft = lpSum([assignment[c][a] for a in aircraft_wp])
    total_cost += calculate_assignment_cost(num_assigned_aircraft)

# Add the additional cost components to the total cost
for c in certifiers:
    for a in aircraft_wp:
        total_cost += calculate_certification_cost(c, a)
        total_cost += calculate_shift_leader_cost(c)
        total_cost += calculate_quality_control_cost(c)

# Add the total cost to the problem
problem += total_cost, "Minimize_Total_Cost"

# Constraints
# Each aircraft should be assigned to exactly one certifier
for a in aircraft_wp:
    problem += lpSum([assignment[c][a] for c in certifiers]) == 1

# Each certifier should be assigned to a maximum of 5 aircraft, if possible
for c in certifiers:
    problem += lpSum([assignment[c][a] for a in aircraft_wp]) <= 5

# Certifiers should be assigned to aircraft they have certification, as Main Certification or Cat-A certification. If they don't have it, they can't be assigned
for c in certifiers:
    for a in aircraft_wp:
        if (aircraft_info[a]['Aircraft type'] not in certifier_info[c]['Main Certifications'] and
            aircraft_info[a]['Aircraft type'] not in certifier_info[c]['Cat-A Certifications']):
            problem += assignment[c][a] == 0

# Add the constraint that certifiers should be assigned to aircraft in their shift range + 1 hour
for c in certifiers:
    for a in aircraft_wp:
        arrival_time = aircraft_info[a]['Arrival time']
        departure_time = aircraft_info[a]['Departure time']

        shift_start_buffered = certifier_info[c]['Shift Start'] - timedelta(hours=1)
        shift_end_buffered = certifier_info[c]['Shift End'] + timedelta(hours=1)

        # Check if the aircraft's arrival or departure time falls within the buffered shift range
        if not (shift_start_buffered <= arrival_time <= shift_end_buffered or
                shift_start_buffered <= departure_time <= shift_end_buffered):
            problem += assignment[c][a] == 0

# Certifiers should be assigned to a maximum of 2 aircraft with long layovers
for c in certifiers:
    problem += lpSum([assignment[c][a] for a in aircraft_wp if aircraft_info[a]['Ground Time (minutes)'] > long_stop]) <= 2

# Link movement variables with assignments
for c in certifiers:
    for a1 in aircraft_wp:
        for a2 in aircraft_wp:
            if a1 != a2:
                problem += movement[c][a1][a2] >= assignment[c][a1] + assignment[c][a2] - 1
                problem += movement[c][a1][a2] <= assignment[c][a1]
                problem += movement[c][a1][a2] <= assignment[c][a2]

# Solve the problem
problem.solve()

# Check the solution
print(f"Status: {LpStatus[problem.status]}")
for c in certifiers:
    for a in aircraft_wp:
        if assignment[c][a].varValue == 1:
            print(f"Certifier {c} is assigned to Aircraft {a}")

Status: Optimal
Certifier A.OBAID is assigned to Aircraft AEC/L-150724-2
Certifier A.SHUAIBI is assigned to Aircraft EIX/L-150724-2
Certifier A.SHUAIBI is assigned to Aircraft AEI/L-150724-2
Certifier SANDEEP is assigned to Aircraft EIH/L-150724-3
Certifier AMIN S is assigned to Aircraft EJA/L-150724-2
Certifier AMIN S is assigned to Aircraft AEE/L-150724-2
Certifier ZAHID is assigned to Aircraft EIW/L-150724-2
Certifier ZAHID is assigned to Aircraft EIY/L-150724-2
Certifier ADIL is assigned to Aircraft EIT/L-150724-2
Certifier ADIL is assigned to Aircraft EJA/L-150724-3
Certifier MITSIS is assigned to Aircraft AEJ/L-150724-2
Certifier NASEERUDDIN is assigned to Aircraft AEF/L-150724-2
Certifier NASEERUDDIN is assigned to Aircraft EII/L-150724-4
Certifier OSMAN is assigned to Aircraft EIN/L-150724-3
Certifier OSMAN is assigned to Aircraft EIU/L-150724-3
Certifier OSMAN is assigned to Aircraft AEG/L-1G0724-2
Certifier JAHANGIR is assigned to Aircraft EIV/L-150724-2
Certifier MARK C. is 

In [14]:
# Print the costs
for c in certifiers:
    num_aircraft_assigned = sum(assignment[c][a].varValue for a in aircraft_wp)
    assignment_cost = calculate_assignment_cost(num_aircraft_assigned)
    print(f"Certifier {c}:")
    print(f"  Number of aircraft assigned: {num_aircraft_assigned}")
    print(f"  Assignment cost: {assignment_cost}")

    movement_cost = 0
    for a1 in aircraft_wp:
        for a2 in aircraft_wp:
            if a1 != a2:
                movement_cost += (
                    movement_cost_matrix[(a1, a2)]
                    * assignment[c][a1].varValue
                    * assignment[c][a2].varValue
                )
    print(f"  Movement cost: {movement_cost}")

    for a in aircraft_wp:
        if assignment[c][a].varValue == 1:
            certification_cost = calculate_certification_cost(c, a)
            print(f"  Aircraft {a}:")
            print(f"    Certification cost: {certification_cost}")

# Print total cost
total_cost_value = value(problem.objective)
print(f"Total cost: {total_cost_value}")

# Print assignments
for c in certifiers:
    for a in aircraft_wp:
        if assignment[c][a].varValue == 1:
            print(f"Certifier {c} is assigned to Aircraft {a}")

Certifier A.OBAID:
  Number of aircraft assigned: 1.0
  Assignment cost: 100.0
  Movement cost: 0.0
  Aircraft AEC/L-150724-2:
    Certification cost: 0
Certifier A.SHUAIBI:
  Number of aircraft assigned: 2.0
  Assignment cost: 200.0
  Movement cost: 0.0
  Aircraft EIX/L-150724-2:
    Certification cost: 0
  Aircraft AEI/L-150724-2:
    Certification cost: 0
Certifier SANDEEP:
  Number of aircraft assigned: 1.0
  Assignment cost: 100.0
  Movement cost: 0.0
  Aircraft EIH/L-150724-3:
    Certification cost: 0
Certifier AMIN S:
  Number of aircraft assigned: 2.0
  Assignment cost: 200.0
  Movement cost: 0.0
  Aircraft EJA/L-150724-2:
    Certification cost: 0
  Aircraft AEE/L-150724-2:
    Certification cost: 0
Certifier CRAIG:
  Number of aircraft assigned: 0.0
  Assignment cost: 0.0
  Movement cost: 0.0
Certifier ZAHID:
  Number of aircraft assigned: 2.0
  Assignment cost: 200.0
  Movement cost: 0.0
  Aircraft EIW/L-150724-2:
    Certification cost: 0
  Aircraft EIY/L-150724-2:
    Cer

In [15]:
# Check the solution status
status = LpStatus[problem.status]
print(f"Status: {status}")

if status == "Infeasible":
    print("The problem is infeasible. Analyzing constraints...")

    # Iterate over constraints to find those that are not satisfied
    for name, constraint in problem.constraints.items():
        if constraint.value() > constraint.constant:
            print(f"Constraint {name} is not satisfied.")
            print(f" - Expression: {constraint}")
            print(f" - Left-hand side value: {constraint.value()}")
            print(f" - Right-hand side value: {constraint.constant}")


# Check the solution (if feasible or optimal)
if status in ["Optimal", "Feasible"]:
    for c in certifiers:
        for a in aircraft_wp:
            if assignment[c][a].varValue == 1:
                print(f"Certifier {c} is assigned to Aircraft {a}")

Status: Optimal
Certifier A.OBAID is assigned to Aircraft AEC/L-150724-2
Certifier A.SHUAIBI is assigned to Aircraft EIX/L-150724-2
Certifier A.SHUAIBI is assigned to Aircraft AEI/L-150724-2
Certifier SANDEEP is assigned to Aircraft EIH/L-150724-3
Certifier AMIN S is assigned to Aircraft EJA/L-150724-2
Certifier AMIN S is assigned to Aircraft AEE/L-150724-2
Certifier ZAHID is assigned to Aircraft EIW/L-150724-2
Certifier ZAHID is assigned to Aircraft EIY/L-150724-2
Certifier ADIL is assigned to Aircraft EIT/L-150724-2
Certifier ADIL is assigned to Aircraft EJA/L-150724-3
Certifier MITSIS is assigned to Aircraft AEJ/L-150724-2
Certifier NASEERUDDIN is assigned to Aircraft AEF/L-150724-2
Certifier NASEERUDDIN is assigned to Aircraft EII/L-150724-4
Certifier OSMAN is assigned to Aircraft EIN/L-150724-3
Certifier OSMAN is assigned to Aircraft EIU/L-150724-3
Certifier OSMAN is assigned to Aircraft AEG/L-1G0724-2
Certifier JAHANGIR is assigned to Aircraft EIV/L-150724-2
Certifier MARK C. is 